# Questions: What are the best features to use when predicting totalGold? What are the best features to use when predicting level without using xp?

In [232]:
import numpy as np
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import json
import pylab
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline

# Import Data

In [3]:
df3 = pd.read_json("back2.json")
df3

,data
0,{'frames': [{'participantFrames': {'1': {'part...
1,{'frames': [{'participantFrames': {'1': {'part...
2,{'frames': [{'participantFrames': {'1': {'part...
3,{'frames': [{'participantFrames': {'1': {'part...
4,{'frames': [{'participantFrames': {'1': {'part...
...,...
993,{'frames': [{'participantFrames': {'1': {'part...
994,{'frames': [{'participantFrames': {'1': {'part...
995,{'frames': [{'participantFrames': {'1': {'part...
996,{'frames': [{'participantFrames': {'1': {'part...


In [ ]:
df4 = pd.DataFrame(columns=['dataId', 'frameNo', 'participantId', 'currentGold', 'totalGold', 'level', 'xp', 'minionsKilled', 'jungleMinionsKilled'])
did = 0
fid = 0
for a in df3['data']:
    for b in a['frames']:
        for c in b['participantFrames']:
            df4 = df4.append({'dataId' : did, 'frameNo' : fid, 'participantId' : b['participantFrames'][c]['participantId'], 'currentGold' : b['participantFrames'][c]['currentGold'], 'totalGold' : b['participantFrames'][c]['totalGold'], 'level' : b['participantFrames'][c]['level'], 'xp' : b['participantFrames'][c]['xp'], 'minionsKilled' : b['participantFrames'][c]['minionsKilled'], 'jungleMinionsKilled' : b['participantFrames'][c]['jungleMinionsKilled']}, ignore_index=True)
df4

# Clean Data

In [233]:
df4["totalGold"] = df4["totalGold"].astype('int64') #convert data to int64's and add totalMinionsKilled column
df4["level"] = df4["level"].astype('int64')
df4["xp"] = df4["xp"].astype('int64')
df4["minionsKilled"] = df4["minionsKilled"].astype('int64')
df4["jungleMinionsKilled"] = df4["jungleMinionsKilled"].astype('int64')
df4["totalMinionsKilled"] = df4["jungleMinionsKilled"] + df4["minionsKilled"]
df4.dtypes

dataId                 object
frameNo                object
participantId          object
currentGold            object
totalGold               int64
level                   int64
xp                      int64
minionsKilled           int64
jungleMinionsKilled     int64
totalMinionsKilled      int64
dtype: object

# Training and Testing Sets

In [205]:
df4 = df4.sample(frac=1) #randomizes the values
ten = int(len(df4.index) * .1) #gets 10% of the data for splitting between training and test sets
df_train = df4[:ten]
df_test = df4[ten:]

In [206]:
df_train.head(5)

,dataId,frameNo,participantId,currentGold,totalGold,level,xp,minionsKilled,jungleMinionsKilled,totalMinionsKilled
45646,161,1,7,500,500,1,0,0,0,0
137554,499,14,5,535,4235,9,5683,83,0,83
254294,936,4,5,969,3069,5,2210,4,0,4
195092,716,16,5,696,7204,12,8656,130,4,134
6646,23,14,6,160,4035,8,4296,3,61,64


In [207]:
df_test.head(5)

,dataId,frameNo,participantId,currentGold,totalGold,level,xp,minionsKilled,jungleMinionsKilled,totalMinionsKilled
116899,423,1,10,1400,1400,1,660,0,0,0
72292,258,5,4,133,1183,4,1385,21,0,21
96544,348,24,5,195,9845,13,10825,147,8,155
115840,419,9,5,612,2712,7,3831,57,0,57
246528,908,6,9,455,3655,6,3073,3,0,3


# Predicting Gold

In [208]:
X_train = df_train.copy(deep=True)
X_train = X_train[["level"]]
X_test = df_test.copy(deep=True)
X_test = X_test[["level"]]
y_train = df_train["totalGold"]
y_test = df_test["totalGold"]
model = LinearRegression()
model.fit(X=X_train, y=y_train)
Y_predict = model.predict(X=X_train)
model.coef_

array([846.84885542])

It seems for every level there is about 846 gold income. At level 18 (the max level) any player will have about 15,000 gold which is around 5 items which makes sense.

In [209]:
pipeline = make_pipeline(StandardScaler(), SGDRegressor())
pipeline.fit(X=X_train, y=y_train)
scores = cross_val_score(pipeline, X=X_train, y=y_train, cv=10)
scores

array([0.85882305, 0.86171407, 0.8666114 , 0.86770675, 0.87125931,
       0.87002529, 0.87040529, 0.86233782, 0.86770266, 0.85735989])

In [210]:
scores.mean()

0.8653945522760399

In [211]:
from sklearn.neighbors import KNeighborsRegressor
temp = KNeighborsRegressor(n_neighbors = 3)
temp.fit(X_train, y_train)
temp.score(X_test, y_test)

0.8673694088485094

In [212]:
X_train = df_train.copy(deep=True)
X_train = X_train[["level", 'minionsKilled', "jungleMinionsKilled"]]
X_test = df_test.copy(deep=True)
X_test = X_test[["level", 'minionsKilled', "jungleMinionsKilled"]]
y_train = df_train["totalGold"]
y_test = df_test["totalGold"]
model = LinearRegression()
model.fit(X=X_train, y=y_train)
Y_predict = model.predict(X=X_train)
model.coef_

array([775.6811595,   6.1681976,  11.2586585])

Adding the importance of minionsKilled and jungleMinionsKilled decreases the importance of level in predicting totalGold. Level will change totalGold less because for each minionsKilled and jungleMinionsKilled the gold value increases.

In [213]:
pipeline = make_pipeline(StandardScaler(), SGDRegressor())
pipeline.fit(X=X_train, y=y_train)
scores = cross_val_score(pipeline, X=X_train, y=y_train, cv=10)
scores

array([0.86578089, 0.86694473, 0.87259903, 0.87236459, 0.87645978,
       0.87540683, 0.87497875, 0.86980033, 0.87308182, 0.86273382])

In [214]:
scores.mean()

0.8710150594205792

In [215]:
from sklearn.neighbors import KNeighborsRegressor
temp = KNeighborsRegressor(n_neighbors = 3)
temp.fit(X_train, y_train)
temp.score(X_test, y_test)

0.902356485312228

In [216]:
X_train = df_train.copy(deep=True)
X_train = X_train[["level", "totalMinionsKilled"]]
X_test = df_test.copy(deep=True)
X_test = X_test[["level", "totalMinionsKilled"]]
y_train = df_train["totalGold"]
y_test = df_test["totalGold"]
model = LinearRegression()
model.fit(X=X_train, y=y_train)
Y_predict = model.predict(X=X_train)
model.coef_

array([774.96223903,   7.19913995])

Compared to using minionsKilled and jungleMinionsKilled, totalMinionsKilled has less weight that the other two combined but keeps the weight for level about the same. However, the accuracies are about the same


In [217]:
pipeline = make_pipeline(StandardScaler(), SGDRegressor())
pipeline.fit(X=X_train, y=y_train)
scores = cross_val_score(pipeline, X=X_train, y=y_train, cv=10)
scores

array([0.86366941, 0.86561795, 0.87195047, 0.8711079 , 0.87518763,
       0.87442019, 0.87440538, 0.86816714, 0.87176525, 0.86139265])

In [218]:
scores.mean()

0.8697683974759416

In [219]:
from sklearn.neighbors import KNeighborsRegressor
temp = KNeighborsRegressor(n_neighbors = 3)
temp.fit(X_train, y_train)
temp.score(X_test, y_test)

0.9006487356045036

The difference between predicting totalGold with minions is definitely better than without using minions. Using totalMinionsKilled vs minionsKilled and jungleMinionsKilled does not make too much of a difference either. KNN was always better except on the run using only level the accuracies were very close so mileage may vary depending on the run.

# Predicting Level

In [220]:
X_train = df_train.copy(deep=True)
X_train = X_train[["totalGold"]]
X_test = df_test.copy(deep=True)
X_test = X_test[["totalGold"]]
y_train = df_train["level"]
y_test = df_test["level"]
model = LinearRegression()
model.fit(X=X_train, y=y_train)
Y_predict = model.predict(X=X_train)
model.coef_

array([0.00102206])

Because the total gold will be a large amount and level will be between 1 and 18, the coefficient is very low.

In [221]:
pipeline = make_pipeline(StandardScaler(), SGDRegressor())
pipeline.fit(X=X_train, y=y_train)
scores = cross_val_score(pipeline, X=X_train, y=y_train, cv=10)
scores

array([0.85893533, 0.86192055, 0.86653482, 0.86758851, 0.87098389,
       0.87003382, 0.87037929, 0.86231034, 0.86784064, 0.85788161])

In [222]:
scores.mean()

0.8654408816264088

In [223]:
from sklearn.neighbors import KNeighborsRegressor
temp = KNeighborsRegressor(n_neighbors = 3)
temp.fit(X_train, y_train)
temp.score(X_test, y_test)

0.9143195417886715

In [224]:
X_train = df_train.copy(deep=True)
X_train = X_train[["totalGold", 'minionsKilled', "jungleMinionsKilled"]]
X_test = df_test.copy(deep=True)
X_test = X_test[["totalGold", 'minionsKilled', "jungleMinionsKilled"]]
y_train = df_train["level"]
y_test = df_test["level"]
model = LinearRegression()
model.fit(X=X_train, y=y_train)
Y_predict = model.predict(X=X_train)
model.coef_

array([0.00090524, 0.01185537, 0.00675202])

Because the total gold will be a large amount, minonsKilled will be relatively low, jungleminionsKilled will relatively low, and level will be between 1 and 18, the coefficients are very low. Minions killed however has the largest coefficient probably because it has the most significance.

In [225]:
pipeline = make_pipeline(StandardScaler(), SGDRegressor())
pipeline.fit(X=X_train, y=y_train)
scores = cross_val_score(pipeline, X=X_train, y=y_train, cv=10)
scores

array([0.86989386, 0.87286617, 0.87351388, 0.87901989, 0.88045366,
       0.8794688 , 0.87910849, 0.87063718, 0.87749392, 0.86960695])

In [226]:
scores.mean()

0.875206282025859

In [227]:
from sklearn.neighbors import KNeighborsRegressor
temp = KNeighborsRegressor(n_neighbors = 3)
temp.fit(X_train, y_train)
temp.score(X_test, y_test)

0.9273790785398349

In [228]:
X_train = df_train.copy(deep=True)
X_train = X_train[["totalGold", 'totalMinionsKilled']]
X_test = df_test.copy(deep=True)
X_test = X_test[["totalGold", 'totalMinionsKilled']]
y_train = df_train["level"]
y_test = df_test["level"]
model = LinearRegression()
model.fit(X=X_train, y=y_train)
Y_predict = model.predict(X=X_train)
model.coef_

array([0.00090332, 0.01096426])

Because the total gold will be a large amount, totalMinonsKilled will be medium, and level will be between 1 and 18, the coefficients are very low. totalMinionsKilled however has the largest coefficient probably because it has the most significance.

In [229]:
pipeline = make_pipeline(StandardScaler(), SGDRegressor())
pipeline.fit(X=X_train, y=y_train)
scores = cross_val_score(pipeline, X=X_train, y=y_train, cv=10)
scores

array([0.86824901, 0.87202337, 0.8730647 , 0.87792018, 0.87936861,
       0.87843584, 0.87838522, 0.86972297, 0.87677519, 0.8684813 ])

In [230]:
scores.mean()

0.8742426392073777

In [231]:
from sklearn.neighbors import KNeighborsRegressor
temp = KNeighborsRegressor(n_neighbors = 3)
temp.fit(X_train, y_train)
temp.score(X_test, y_test)

0.9267886159529272

When predicting the level, KNN on average had about 5 more accuracy than is linear regression counterpart. Linear regression averaged around 87% and KNN averaged around 92%. totalGold, minionsKilled, and jungleMinionsKilled was about the same accuracy as totalGold and totalMinionsKilled, which makes sense because you gain xp by being around dying minions and champions. If you have more minions killed then you were more likely to be around minions that are dying. It generally does not matter the difference between jungle minions and regular minions because they give the same generaly amount of xp.